In [ ]:
import openai
import requests
from dotenv import load_dotenv
import os
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import random
import time
import pandas as pd

# Load environment variables
load_dotenv()

# Set up your OpenAI API key from environment variable
client = openai  # Client object to handle API calls
client.api_key = os.getenv('OPENAI_API_KEY')

# Correct folder names (no hyphens)
model_folders = {
    'dalle2': './dalle2',
    'dalle3': './dalle3',
    'gpt-3.5-turbo-0125': './gpt3_5',
    'gpt-4-turbo': './gpt4'
}

# Create directories if they don't exist
for folder in model_folders.values():
    os.makedirs(folder, exist_ok=True)

# Ensure default folder is created if needed
default_folder = './default_folder'
os.makedirs(default_folder, exist_ok=True)

# Function to generate image and handle errors
def generate_image_from_model(model_name, prompt, file_name, model_folder):
    try:
        print(f"Generating image for model: '{model_name}', prompt: '{prompt}', saving as: {file_name} in {model_folder}")
        
        if model_name == 'dalle2':
            response = client.images.generate(
                prompt=prompt,
                n=1,  # Number of images
                size="1024x1024"
            )
        elif model_name == 'dalle3':
            response = client.images.generate(
                prompt=prompt,
                n=1,  # Number of images
                size="1024x1024",
                quality="hd"
            )
        else:
            # For GPT models, skip image generation
            return None  # No action for GPT-3.5 and GPT-4
        
        # Ensure the response has the data field and an image URL
        if 'data' in response and len(response['data']) > 0 and 'url' in response['data'][0]:
            image_url = response['data'][0]['url']
            print(f"Image URL: {image_url}")

            # Use the OpenAI client to retrieve and handle the image file
            image_data = requests.get(image_url).content

            # Save the image file locally
            img = Image.open(BytesIO(image_data))
            img = img.convert("RGB")  # Convert PNG to RGB for saving as JPG

            img_path = f'{model_folder}/{file_name}.jpg'
            img.save(img_path, 'JPEG', dpi=(300, 300))
            print(f"Image saved successfully as: {img_path}")
            return None  # No error
        else:
            return "No image URL found in API response."
        
    except Exception as e:
        print(f"Failed to generate image: {e}")
        return str(e)  # Return error message

# Main function to read from the Excel file and process each row
def process_image_prompts(excel_file):
    # Load the Excel file
    df = pd.read_excel(excel_file)

    # Check if an error column exists, if not, add it
    if 'error_no' not in df.columns:
        df['error_no'] = ''

    # Iterate through each row in the Excel file
    for idx, row in tqdm(df.iterrows(), desc="Processing prompts", total=len(df)):
        model_name = row['model']
        file_name = row['file_name']
        error_no = row['error_no']

        # If there's already an error recorded, skip processing
        if pd.notna(error_no) and error_no != '':
            print(f"Skipping {file_name} due to previous error: {error_no}")
            continue

        # Select the appropriate prompt based on the model
        if model_name == 'dalle2':
            prompt = row['prompt_300']
        elif model_name == 'dalle3':
            prompt = row['prompt_3k']
        else:
            prompt = row['prompt']  # For GPT-3.5 and GPT-4

        # Get the folder for the model, use default folder if model name is unrecognized
        model_folder = model_folders.get(model_name, default_folder)

        # Generate image and check for errors
        error_message = generate_image_from_model(model_name, prompt, file_name, model_folder)

        # Record the error if any and skip the wait
        if error_message:
            df.at[idx, 'error_no'] = error_message
        else:
            # Add a random wait time between image generations to avoid API limits
            delay = random.randint(30, 90)
            print(f"Waiting for {delay} seconds before the next image...")
            time.sleep(delay)

    # Save the updated DataFrame back to the Excel file
    df.to_excel(excel_file, index=False)
    print(f"Excel file updated with any errors: {excel_file}")

# Path to the Excel file
excel_file = '../datasets/image_prompts.xlsx'

# Process the image prompts
process_image_prompts(excel_file)


Processing prompts:   0%|          | 0/396 [00:00<?, ?it/s]

Generating image for model: 'dalle2', prompt: 'Busy Urban Intersection at Rush Hour: Picture the vibrant energy of a bustling city intersection during rush hour, with streams of cars, buses, and bicycles weaving through the traffic. Pedestrians of diverse backgrounds hurry across crosswalks, while towering skyscrapers reflect the golden hues of ', saving as: G02_I_urban_0001 in ./dalle2


Processing prompts:   1%|          | 2/396 [00:00<02:06,  3.11it/s]

Failed to generate image: Error code: 400 - {'error': {'code': 'billing_hard_limit_reached', 'message': 'Billing hard limit has been reached', 'param': None, 'type': 'invalid_request_error'}}
Generating image for model: 'dalle3', prompt: 'Busy Urban Intersection at Rush Hour: Picture the vibrant energy of a bustling city intersection during rush hour, with streams of cars, buses, and bicycles weaving through the traffic. Pedestrians of diverse backgrounds hurry across crosswalks, while towering skyscrapers reflect the golden hues of the setting sun. The cacophony of honking horns, distant sirens, and the murmur of the crowd captures the unceasing pulse of urban life.', saving as: G03_I_urban_0001 in ./dalle3
Failed to generate image: Error code: 400 - {'error': {'code': 'billing_hard_limit_reached', 'message': 'Billing hard limit has been reached', 'param': None, 'type': 'invalid_request_error'}}
Generating image for model: 'gpt3.5-turbo-0125', prompt: 'Busy Urban Intersection at Rush 